## Modelele necesare pentru generarea de feedback-uri

In [1]:
import psycopg2
from datetime import datetime


class User:
    def __init__(self, user_id):
        self.user_id = user_id


class Place:
    def __init__(self, place_id):
        self.place_id = place_id

class Feedback:
    def __init__(self, text, stars, user, place):
        self.feedback_id = None  # Va fi generat automat
        self.text = text
        self.stars = stars
        self.user = user
        self.place = place


    def save_to_database(self, conn):
        cursor = conn.cursor()

        # Inserare feedback în baza de date
        insert_query = "INSERT INTO feedbacks (feedback_text, stars, user_id, place_id) VALUES (%s, %s, %s, %s) RETURNING feedback_id"
        try:
            cursor.execute(insert_query, (self.text, str(self.stars), str(self.user.user_id), str(self.place.place_id)))
            self.feedback_id = cursor.fetchone()[0]
            conn.commit()
            print("Feedback inserat cu succes!")
        except psycopg2.IntegrityError:
            conn.rollback()
            print("Eroare: Perechea user_id și place_id este duplicată!")

        cursor.close()

class Visit:
    def __init__(self, user, place):
        self.visit_id = None
        self.user = user
        self.place = place
        self.visited_at = datetime.now()

    def save_to_database(self, conn):
        cursor = conn.cursor()

        # Inserare feedback în baza de date
        insert_query = "INSERT INTO visits (user_id, place_id, visited_at) VALUES (%s, %s, %s) RETURNING visit_id"
        try:
            cursor.execute(insert_query, (self.user.user_id, self.place.place_id, self.visited_at))
            self.visit_id = cursor.fetchone()[0]
            conn.commit()
            print("Vizita inserata cu succes!")
        except psycopg2.IntegrityError:
            conn.rollback()
            print("Eroare: Perechea user_id și place_id este duplicată!")

        cursor.close()

## Modelul cu care vom genera textul pentru feedback-uri

In [2]:
from transformers import pipeline

text_generation = pipeline('text-generation', model='gpt2-large')
def generate_text_with_prefix(prefix: str):
    generated_text = text_generation(prefix, max_length=50, do_sample=False)[0]
    return generated_text['generated_text']

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.11.3 (you have 3.11.2)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [12]:
generate_text_with_prefix("5 star review by mami: Brasov's tourist attraction Black Church is")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"5 star review by mami: Brasov's tourist attraction Black Church is a beautiful and unique place. It is a place of worship and a place of prayer. It is a place of peace and a place of prayer. It is a place of"

## Generarea de feedback-uri

In [3]:
import random
from translate import Translator


# Generare feedback-uri aleatorii
feedbacks_count = 10
conn = psycopg2.connect(
        host="localhost",
        database="places",
        user="postgres",
        password="postgres"
    )
for i in range(feedbacks_count):
    translator = Translator(from_lang='romanian', to_lang='english')
    translatorRo = Translator(from_lang='english', to_lang='romanian')
    cursor = conn.cursor()
    stars = random.randint(1, 5)

    # Obținere utilizator aleatoriu
    user_query = "SELECT id, username FROM _user ORDER BY RANDOM() LIMIT 1"
    cursor.execute(user_query)
    [user_id, username] = cursor.fetchone()

    # Obținere loc aleatoriu
    place_query = "SELECT p.place_id, p.name FROM places p LEFT JOIN feedbacks f ON p.place_id = f.place_id GROUP BY p.place_id, p.name HAVING COUNT(f.*) <= ALL (SELECT COUNT(f.*) FROM places p LEFT JOIN feedbacks f ON p.place_id = f.place_id GROUP BY p.place_id) ORDER BY RANDOM() LIMIT 1"
    cursor.execute(place_query)
    [place_id, place_name] = cursor.fetchone()

    start = f"{stars} review by {username}: Brasov, Romania, tourist attraction "
    # place_name = translator.translate(place_name)
    text = generate_text_with_prefix(start + place_name + ' is')
    text = text[len(start):]
    text = '.'.join(text.split('.')[:2])
    if random.random() > 0.75:
        text = translatorRo.translate(text)
    if len(text) > 240:
        text = text[:240]
    print(text)

    feedback = Feedback(text, stars, User(int(user_id)), Place(int(place_id)))
    feedback.save_to_database(conn)
    cursor.close()

conn.close()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Paradisul Acvatic is a beautiful and unique place. It is a place where you can see the sun rise and set, and the sun rises and sets in the same place
Feedback inserat cu succes!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Diferența este mică, dar esențială, iar rezultatul este vizibil după un timp relativ scurt.
Feedback inserat cu succes!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Belvedere Brașov is a beautiful and unique attraction in the Romanian capital. It is a large, circular, circular, circular, circular, circular, circular, circular, circular
Feedback inserat cu succes!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Castelul Peleș is a beautiful and unique attraction in the city of Brasov, Romania. It is a small, but very interesting, attraction
Feedback inserat cu succes!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Parcul Central is a beautiful and unique place. It is a very popular tourist attraction and is a great place to visit
Feedback inserat cu succes!
Dino Parc Rasnov is a small, but very interesting, attraction. It is located in the town of Brasov, Romania, and is a small, but very interesting, attraction
Feedback inserat cu succes!


In [10]:
# inseram vizite

visits_count = 95
conn = psycopg2.connect(
        host="localhost",
        database="places",
        user="postgres",
        password="postgres"
    )
cursor = conn.cursor()
for _ in range(visits_count):
    # Obținere utilizator aleatoriu
    user_query = "SELECT id FROM _user ORDER BY RANDOM() LIMIT 1"
    cursor.execute(user_query)
    user_id = cursor.fetchone()[0]

    # Obținere loc aleatoriu
    place_query = "SELECT place_id FROM places ORDER BY RANDOM() LIMIT 1"
    cursor.execute(place_query)
    place_id = cursor.fetchone()[0]

    visit = Visit(User(user_id), Place(place_id))
    visit.save_to_database(conn)
cursor.close()
conn.close()

Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Eroare: Perechea user_id și place_id este duplicată!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vizita inserata cu succes!
Vi

## Actualizare date

In [ ]:
conn = psycopg2.connect(
        host="localhost",
        database="places",
        user="postgres",
        password="postgres"
    )
cursor = conn.cursor()
update_query = """ 
    UPDATE _user U2
    SET personal_score = (
        SELECT SUM(p.points) AS points FROM _user U
            INNER JOIN visits v on U.id = v.user_id
            INNER JOIN places p ON v.place_id = p.place_id
            WHERE U2.id = U.id) + (
        SELECT COUNT(F) AS feedback FROM _user U
            INNER JOIN feedbacks F on U.id = F.user_id
            WHERE U2.id = U.id)
 """
cursor.execute(update_query)
conn.commit()
cursor.close()
conn.close()